# Does your dorm matter for your well-being?

We build models to predict:
1. Spring well-being from fall well-being
1. Spring well-being from fall well-being, demographic items (age, family income, family education, race, gender), and ambient empathy
1. Same, plus random effects by dorm.

# Results:
- Demographics and ambient empathy do not improve model
- Random effect model does not improve fit, and no variance is apportioned to the dorm level

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Configuration" data-toc-modified-id="Configuration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Configuration</a></span></li><li><span><a href="#Import-and-load" data-toc-modified-id="Import-and-load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import and load</a></span></li><li><span><a href="#Quick-summary-of-whole-dorm-well-beings" data-toc-modified-id="Quick-summary-of-whole-dorm-well-beings-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Quick summary of whole-dorm well-beings</a></span></li><li><span><a href="#Standard-regression-models-(not-mixed)" data-toc-modified-id="Standard-regression-models-(not-mixed)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Standard regression models (not mixed)</a></span><ul class="toc-item"><li><span><a href="#Base-model,-minimal-predictors" data-toc-modified-id="Base-model,-minimal-predictors-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Base model, minimal predictors</a></span></li><li><span><a href="#Add-demographic-covariates" data-toc-modified-id="Add-demographic-covariates-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Add demographic covariates</a></span></li><li><span><a href="#Is-this-a-significant-improvement?-(No)" data-toc-modified-id="Is-this-a-significant-improvement?-(No)-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Is this a significant improvement? (No)</a></span></li></ul></li><li><span><a href="#Mixed-effect-models" data-toc-modified-id="Mixed-effect-models-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Mixed effect models</a></span><ul class="toc-item"><li><span><a href="#REML-model-to-accurately-determine-variance-apportioned-to-dorm-(zero)" data-toc-modified-id="REML-model-to-accurately-determine-variance-apportioned-to-dorm-(zero)-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>REML model to accurately determine variance apportioned to dorm (zero)</a></span></li><li><span><a href="#REML=false-model-to-maximize-predictive-value.-Is-this-a-significant-improvement-over-the-non-mixed-model?-(No)" data-toc-modified-id="REML=false-model-to-maximize-predictive-value.-Is-this-a-significant-improvement-over-the-non-mixed-model?-(No)-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>REML=false model to maximize predictive value. Is this a significant improvement over the non-mixed model? (No)</a></span></li></ul></li><li><span><a href="#Bring-in-network-density-to-the-mixed-model" data-toc-modified-id="Bring-in-network-density-to-the-mixed-model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Bring in network density to the mixed model</a></span><ul class="toc-item"><li><span><a href="#Prepare-the-data" data-toc-modified-id="Prepare-the-data-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Prepare the data</a></span></li><li><span><a href="#Mixed-models---REML" data-toc-modified-id="Mixed-models---REML-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Mixed models - REML</a></span></li></ul></li></ul></div>

## Configuration

In [27]:
DATA_FILE = 'data/postprocessed/final_for_analysis_R.csv'

IMPUTE_MISSING = TRUE
INCLUDE_FALL_WB_AS_PREDICTOR = TRUE
INCLUDE_DEMOS_AS_PREDICTOR = TRUE
# DV = 'Wellbeing_fall'
DV = 'Wellbeing_spring'

if (INCLUDE_FALL_WB_AS_PREDICTOR) {
    stopifnot(DV == 'Wellbeing_spring')
}

## Import and load

In [28]:
library(car)
library(plyr)
library(tidyverse)
library(hexbin)
library(mice)
library(nlme)
library(lme4)
library(lmerTest)

options(width=200)

In [29]:
df = read.csv(DATA_FILE, na.strings=c("", " ", "NA"))
df = df[,c('NID', 'Age', 'ParentEducationMax',
           'FinclAid', 'FmlyIncome', 'Gender', 'Race',
           'Ambient_empathy',
           'Wellbeing_fall', 'Wellbeing_spring')]
dim(df)
head(df)

[1] 204  10

,NID,Age,ParentEducationMax,FinclAid,FmlyIncome,Gender,Race,Ambient_empathy,Wellbeing_fall,Wellbeing_spring
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,7,18,4.0,0,87500,M,white,-0.7154534,-2.06354788,-0.76535414
2,11,18,3.5,1,NA,F,south_asian,-0.8199099,-0.01143413,-0.04997158
3,9,18,4.0,1,125000,M,white,-0.8994971,0.91965600,0.66541099
4,4,18,4.0,0,200000,F,east_asian,NA,0.65342017,0.48656535
5,5,18,2.5,1,125000,M,south_asian,-0.4873343,0.69839160,-0.04997158
6,13,18,4.0,1,45000,F,east_asian,NA,0.04290417,-0.13939440


In [30]:
if (IMPUTE_MISSING) {
    print("Imputing missing values")
    imp = mice(df)
    df = complete(imp)
    head(df)
} else {
    df = na.omit(df)
}

[1] "Imputing missing values"

 iter imp variable
  1   1  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  1   2  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  1   3  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  1   4  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  1   5  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  2   1  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  2   2  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  2   3  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  2   4  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  2   5  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  3   1  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  3   2  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
  3   3  ParentEducationMax  FinclAid  FmlyIncome  Race  Ambient_empathy
 

,NID,Age,ParentEducationMax,FinclAid,FmlyIncome,Gender,Race,Ambient_empathy,Wellbeing_fall,Wellbeing_spring
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,7,18,4.0,0,87500,M,white,-0.7154534,-2.06354788,-0.76535414
2,11,18,3.5,1,200000,F,south_asian,-0.8199099,-0.01143413,-0.04997158
3,9,18,4.0,1,125000,M,white,-0.8994971,0.91965600,0.66541099
4,4,18,4.0,0,200000,F,east_asian,-1.1311982,0.65342017,0.48656535
5,5,18,2.5,1,125000,M,south_asian,-0.4873343,0.69839160,-0.04997158
6,13,18,4.0,1,45000,F,east_asian,-0.8199099,0.04290417,-0.13939440


## Quick summary of whole-dorm well-beings

In [31]:
df %>% group_by(NID) %>%
    summarize(wb_fall = mean(Wellbeing_fall),
              wb_spring = mean(Wellbeing_spring))

NID,wb_fall,wb_spring
<dbl>,<dbl>,<dbl>
1,0.17958135,0.43067609
2,0.10039586,0.23049091
4,-0.04213750,0.00871215
5,0.05322856,0.01841058
7,0.06902635,0.17164150
8,-0.15094904,-0.45876161
9,-0.09749154,-0.34804765
10,-0.35117851,-0.30147326
11,0.14734051,0.15272015


## Standard regression models (not mixed)

### Base model, minimal predictors

In [32]:
equation = paste(DV, ' ~  1')
if (INCLUDE_FALL_WB_AS_PREDICTOR) {
    equation = paste(equation, ' + Wellbeing_fall')
}
print(equation)
model1 = lm(as.formula(equation), df)
summary(model1)

[1] "Wellbeing_spring  ~  1  + Wellbeing_fall"



Call:
lm(formula = as.formula(equation), data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.69297 -0.48662  0.06646  0.49158  2.76843 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)    -3.286e-16  5.373e-02    0.00        1    
Wellbeing_fall  6.433e-01  5.387e-02   11.94   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7675 on 202 degrees of freedom
Multiple R-squared:  0.4139,	Adjusted R-squared:  0.411 
F-statistic: 142.6 on 1 and 202 DF,  p-value: < 2.2e-16


### Add demographic covariates

In [33]:
names(df)

[1] "NID"                "Age"                "ParentEducationMax" "FinclAid"           "FmlyIncome"         "Gender"             "Race"               "Ambient_empathy"    "Wellbeing_fall"    
[10] "Wellbeing_spring"

In [34]:
if (INCLUDE_DEMOS_AS_PREDICTOR) {
    equation = paste(equation, '+ Age + ParentEducationMax + FinclAid + FmlyIncome + Gender + Race + Ambient_empathy')
    print(equation)
    model2 = lm(as.formula(equation), df)
    summary(model2)
} else {
    model2 = model1
}

[1] "Wellbeing_spring  ~  1  + Wellbeing_fall + Age + ParentEducationMax + FinclAid + FmlyIncome + Gender + Race + Ambient_empathy"



Call:
lm(formula = as.formula(equation), data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.49880 -0.38430  0.02461  0.50533  2.50432 

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)        -4.394e-01  1.037e+00  -0.424 0.672155    
Wellbeing_fall      6.076e-01  5.513e-02  11.020  < 2e-16 ***
Age                -6.803e-04  5.173e-02  -0.013 0.989521    
ParentEducationMax  5.868e-02  9.571e-02   0.613 0.540506    
FinclAid            1.559e-02  1.380e-01   0.113 0.910148    
FmlyIncome          1.681e-06  9.990e-07   1.683 0.094072 .  
GenderM             1.868e-01  1.115e-01   1.674 0.095698 .  
Genderother         3.684e-02  3.539e-01   0.104 0.917204    
Raceeast_asian      1.137e-01  2.192e-01   0.519 0.604486    
Racehispanic        3.246e-01  2.745e-01   1.183 0.238473    
Raceother_or_mixed -5.437e-02  2.183e-01  -0.249 0.803545    
Racesouth_asian    -1.797e-01  2.935e-01  -0.612 0.541205    
Racewhite           2.

### Is this a significant improvement? (No)

In [35]:
anova(model1, model2)

,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,202,118.9815,NA,NA,NA,NA
2,190,105.6852,12,13.29623,1.991988,0.02690039


## Mixed effect models

### REML model to accurately determine variance apportioned to dorm (zero)

In [36]:
model3 = lmer(as.formula(paste(equation, '+ (1|NID)')), data=df, REML=TRUE)
summary(model3)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”
boundary (singular) fit: see ?isSingular

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

Correlation matrix not shown by default, as p = 14 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method ['lmerModLmerTest']
Formula: as.formula(paste(equation, "+ (1|NID)"))
   Data: df

REML criterion at convergence: 503.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3504 -0.5153  0.0330  0.6776  3.3578 

Random effects:
 Groups   Name        Variance Std.Dev.
 NID      (Intercept) 0.0000   0.0000  
 Residual             0.5562   0.7458  
Number of obs: 204, groups:  NID, 11

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)        -4.394e-01  1.037e+00  1.900e+02  -0.424 0.672155    
Wellbeing_fall      6.076e-01  5.513e-02  1.900e+02  11.020  < 2e-16 ***
Age                -6.803e-04  5.173e-02  1.900e+02  -0.013 0.989521    
ParentEducationMax  5.868e-02  9.571e-02  1.900e+02   0.613 0.540506    
FinclAid            1.559e-02  1.380e-01  1.900e+02   0.113 0.910148    
FmlyIncome          1.681e-06  9.990e-07  1.900e+02   1.683 0.094072 .  
GenderM   

### REML=false model to maximize predictive value. Is this a significant improvement over the non-mixed model? (No)

In [37]:
model4 = lmer(as.formula(paste(equation, '+ (1|NID)')), data=df, REML=FALSE)
anova(model4, model2)#, refit=FALSE)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”
boundary (singular) fit: see ?isSingular

Warning message:
“Some predictor variables are on very different scales: consider rescaling”


,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model2,15,474.7653,524.5371,-222.3827,444.7653,NA,NA,NA
model4,16,476.7653,529.8552,-222.3827,444.7653,0,1,1


## Bring in network density to the mixed model

### Prepare the data

In [38]:
density = read.csv('data/NetworkDensity2018.csv')
density = density[,2:ncol(density)]  # First row is a meaningless row number
head(density)

,Dorm,Network,Density
,<fct>,<fct>,<dbl>
1,FroSoCo,SpendTime,0.0008166282
2,Norcliffe&Adelfa,SocAdvice,0.0002515091
3,Meier&Naranja,EmpSupp,0.0002639293
4,FroSoCo,EmpSupp,0.0006054848
5,Okada,Persuasive,0.0001800929
6,JRo,NegAffPres,0.0001459374


In [39]:
table(density$Dorm)


         Alondra            Cedro          FroSoCo              JRo           Larkin    Meier&Naranja Norcliffe&Adelfa            Okada            Twain           Ujamaa        WestFloMo 
              12               12               12               12               12               12               12               12               12               12               12 

In [40]:
density$NID <- mapvalues(
    density$Dorm, 
    from=c("Alondra", "Cedro", "EAST", "FroSoCo", "JRo", "Kimball", "Larkin", "Okada", "Twain", "Ujamaa", "Meier&Naranja", "Norcliffe&Adelfa", "WestFloMo"), 
    to=c("1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "13", "15"))

The following `from` values were not present in `x`: EAST, Kimball



In [41]:
table(density$NID)


 1  2  4  5  7 11 13  8  9 10 15 
12 12 12 12 12 12 12 12 12 12 12 

In [42]:
table(density$Network)


 CloseFrds    EmpSupp     Gossip      Liked NegAffPres NegEmoSupp Persuasive PosAffPres PosEmoSupp Responsive  SocAdvice  SpendTime 
        11         11         11         11         11         11         11         11         11         11         11         11 

In [43]:
density_close_friends = density %>%
    filter(Network == 'CloseFrds') %>%
    select(NID, Density) %>%
    arrange(NID)
names(density_close_friends) = c('NID', 'DensityCloseFriends')
head(density_close_friends)

,NID,DensityCloseFriends
,<fct>,<dbl>
1,1,0.0005430616
2,2,0.0008209813
3,4,0.0009838725
4,5,0.0007545272
5,7,0.0006226650
6,11,0.0003512068


In [44]:
density_bad_news = density %>%
    filter(Network == 'NegEmoSupp') %>%
    select(NID, Density) %>%
    arrange(NID)
names(density_bad_news) = c('NID', 'DensityBadNews')
head(density_bad_news)

,NID,DensityBadNews
,<fct>,<dbl>
1,1,0.0003078994
2,2,0.0007001956
3,4,0.0005728361
4,5,0.0005513034
5,7,0.0003902446
6,11,0.0002327147


In [45]:
head(df)

,NID,Age,ParentEducationMax,FinclAid,FmlyIncome,Gender,Race,Ambient_empathy,Wellbeing_fall,Wellbeing_spring
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,7,18,4.0,0,87500,M,white,-0.7154534,-2.06354788,-0.76535414
2,11,18,3.5,1,200000,F,south_asian,-0.8199099,-0.01143413,-0.04997158
3,9,18,4.0,1,125000,M,white,-0.8994971,0.91965600,0.66541099
4,4,18,4.0,0,200000,F,east_asian,-1.1311982,0.65342017,0.48656535
5,5,18,2.5,1,125000,M,south_asian,-0.4873343,0.69839160,-0.04997158
6,13,18,4.0,1,45000,F,east_asian,-0.8199099,0.04290417,-0.13939440


In [46]:
df = merge(df, density_close_friends, on="NID", all.x=TRUE)
df = merge(df, density_bad_news, on="NID", all.x=TRUE)
df[sample(nrow(df), 5), ]

,NID,Age,ParentEducationMax,FinclAid,FmlyIncome,Gender,Race,Ambient_empathy,Wellbeing_fall,Wellbeing_spring,DensityCloseFriends,DensityBadNews
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
135,9,18,4.0,1,125000,M,white,-0.8994971,0.9196560,0.665411,0.0003469754,0.0002437792
133,9,18,4.0,0,200000,F,east_asian,-1.3111573,0.3062250,0.665411,0.0003469754,0.0002437792
158,11,18,3.5,0,125000,F,east_asian,-0.4701864,-0.8315366,-1.033623,0.0003512068,0.0002327147
108,7,18,3.5,0,200000,M,white,-0.7420122,1.2440654,1.470216,0.0006226650,0.0003902446
179,13,18,2.0,1,5000,M,other_or_mixed,-1.1293960,-0.8432925,-1.212468,0.0003007478,0.0002094053


### Mixed models - REML

In [47]:
model5 = lmer(as.formula(paste(equation, '+ DensityCloseFriends + DensityBadNews + (1|NID)')), data=df, REML=TRUE)
summary(model5)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”
boundary (singular) fit: see ?isSingular

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

Correlation matrix not shown by default, as p = 16 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by REML. t-tests use Satterthwaite's method ['lmerModLmerTest']
Formula: as.formula(paste(equation, "+ DensityCloseFriends + DensityBadNews + (1|NID)"))
   Data: df

REML criterion at convergence: 476

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2905 -0.4945  0.0252  0.6634  3.4058 

Random effects:
 Groups   Name        Variance Std.Dev.
 NID      (Intercept) 0.0000   0.0000  
 Residual             0.5608   0.7489  
Number of obs: 204, groups:  NID, 11

Fixed effects:
                      Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)         -7.706e-01  1.160e+00  1.880e+02  -0.664  0.50736    
Wellbeing_fall       6.057e-01  5.555e-02  1.880e+02  10.904  < 2e-16 ***
Age                  1.266e-02  5.594e-02  1.880e+02   0.226  0.82115    
ParentEducationMax   5.433e-02  9.682e-02  1.880e+02   0.561  0.57538    
FinclAid             1.571e-02  1.385e-01  1.880e+02   0.113  0.90982    
FmlyIncome           1.688e-06  1.003e-0

In [48]:
model6 = lmer(as.formula(paste(equation, '+ DensityCloseFriends + DensityBadNews + (1|NID)')), data=df, REML=FALSE)
summary(model6)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”
boundary (singular) fit: see ?isSingular

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

Correlation matrix not shown by default, as p = 16 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's method ['lmerModLmerTest']
Formula: as.formula(paste(equation, "+ DensityCloseFriends + DensityBadNews + (1|NID)"))
   Data: df

     AIC      BIC   logLik deviance df.resid 
   480.3    540.0   -222.1    444.3      186 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.4277 -0.5151  0.0262  0.6911  3.5478 

Random effects:
 Groups   Name        Variance Std.Dev.
 NID      (Intercept) 0.0000   0.0000  
 Residual             0.5169   0.7189  
Number of obs: 204, groups:  NID, 11

Fixed effects:
                      Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)         -7.706e-01  1.114e+00  2.040e+02  -0.692   0.4898    
Wellbeing_fall       6.057e-01  5.333e-02  2.040e+02  11.359   <2e-16 ***
Age                  1.266e-02  5.371e-02  2.040e+02   0.236   0.8138    
ParentEducationMax   5.433e-02  9.294e-02  2.040e+02   0.585   0.5595    
FinclAid             1.571e-02  1.330e-01 

In [49]:
anova(model6, model2)

,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model2,15,474.7653,524.5371,-222.3827,444.7653,NA,NA,NA
model6,18,480.2881,540.0143,-222.1441,444.2881,0.4771757,3,0.9238764
